# Explore the functionalities of agentpy in the context of the Schelling model

This example is taken from the [unisa github](https://github.com/isislab-unisa/ABM_Comparison/blob/main/AgentPy/schelling.py). Note that agentpy cannot integrate GIS info, contrarily to mesa, and so this is already a difference.

In [1]:
import agentpy as ap

In [2]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

### Define the agent

In [4]:
class Person(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.group = self.random.choice(range(self.p.n_groups))
        self.share_similar = 0
        self.happy = False

    def update_happiness(self):
        """ Be happy if rate of similar neighbors is high enough. """
        neighbors = self.grid.neighbors(self)
        similar = len([n for n in neighbors if n.group == self.group])
        ln = len(neighbors)
        self.share_similar = similar / ln if ln > 0 else 0
        self.happy = self.share_similar >= self.p.want_similar

    def find_new_home(self):
        """ Move to random free spot and update free spots. """
        new_spot = self.random.choice(self.model.grid.empty)
        self.grid.move_to(self, new_spot)

### Define the model

In [5]:
class SegregationModel(ap.Model):

    def setup(self):

        # Parameters
        s = self.p.size
        n = self.n = int(self.p.density * (s ** 2))

        # Create grid and agents
        self.grid = ap.Grid(self, (s, s), track_empty=True)
        self.agents = ap.AgentList(self, n, Person)
        self.grid.add_agents(self.agents, random=True, empty=True)

    def update(self):
        # Update list of unhappy people
        self.agents.update_happiness()
        self.unhappy = self.agents.select(self.agents.happy == False)

        # Stop simulation if all are happy
        if len(self.unhappy) == 0:
            self.stop()

    def step(self):
        # Move unhappy people to new location
        self.unhappy.find_new_home()

    def get_segregation(self):
        # Calculate average percentage of similar neighbors
        return round(sum(self.agents.share_similar) / self.n, 2)

    def end(self):
        # Measure segregation at the end of the simulation
        self.report('segregation', self.get_segregation())

### Run the model

In [6]:
parameters = {
    'want_similar': 0.3, # For agents to be happy
    'n_groups': 2, # Number of groups
    'density': 0.7, # Density of population
    'size': 1131, # Height and length of the grid
    'steps': 200  # Maximum number of steps
    }

In [7]:
exp = ap.Experiment(SegregationModel, parameters, iterations=1, randomize=True)
results = exp.run()

Scheduled runs: 1
Completed: 1, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:02:40.921552


In [9]:
results.keys()

dict_keys(['info', 'parameters', 'reporters'])

In [10]:
results['info']

{'model_type': 'SegregationModel',
 'time_stamp': '2023-02-23 15:11:46.891384',
 'agentpy_version': '0.1.5',
 'python_version': '3.9.1',
 'experiment': True,
 'scheduled_runs': 1,
 'completed': True,
 'random': True,
 'record': False,
 'sample_size': 1,
 'iterations': 1,
 'run_time': '0:02:40.921552'}